# Paso 9. Ecuación de Laplace para la presión

$$
\frac{\partial^2 p}{\partial x^2} + \frac{\partial^2 p}{\partial y^2} = 0
$$

La ecuación de Laplace nos habla de la estabilidad de un sistema, ya que no hay derivadas con respecto al tiempo. Para nuestro caso lo que queremos es dadas unas condiciones iniciales, hallar cual es el profile de estabilidad del sistema haciendolo avanzar hasta que la diferencia normalizada del profile actual con el anterior tienda a cero. $\frac{\big|p(i,j)-p(i,j)_{n+1}\big|}{\big|p(i,j)_{n+1}\big|}$

#### Condiciones iniciales y valores de frontera

Para el dominio $[0, 2]\times[0, 1]$

\begin{eqnarray}
p(x,y) &=& 0\\
p(0,y) &=& 0\\
p(2,y) &=& y\\
\frac{\partial p}{\partial y} &=& 0, \, \, \text{en y=0,1}
\end{eqnarray}
#### Discretización

$$
\frac{p(i+1, j)_n - 2p(i, j)_n + p(i-1, j)_n}{\Delta x^2} + \frac{p(i, j+1)_n - 2p(i, j)_n + p(i, j-1)_n}{\Delta x^2} = 0
$$


$$
p(i,j)_n = \frac{\Delta y^2\Big[p(i+1,j)_n + p(i-1,j)_n\Big] + \Delta x^2\Big[p(i,j+1)_n + p(i,j-1)_n \Big] }{2(\Delta x^2 + \Delta y^2)}
$$

In [4]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

def update(i, profile):
    #store the actual state of the array
    
    p_n = p;

    pn_i_j   = p_n[1:,1:];
    pn_im1_j = np.roll(p,  1, axis=1)[1:, 1:];
    pn_ip1_j = np.roll(p, -1, axis=1)[1:, 1:];
    pn_i_jm1 = np.roll(p,  1, axis=0)[1:, 1:];
    pn_i_jp1 = np.roll(p, -1, axis=0)[1:, 1:];

    p[1:,1:] = ((dy**2)*(pn_ip1_j + pn_im1_j) + (dx**2)*(pn_i_jp1 + pn_i_jm1)) / (2*(dx**2 + dy**2))

    """
    p = 0 at x = 0
    p = y at x = 2
    dp/dx = 0 at y = 0 and y = 1
    """
    p[:,0] = 0;
    p[:,-1] = y;
    p[0,:] = p[1,:];
    p[-1,:] = p[-2,:];    

    ax.clear();
    ax.grid(False);

    profile = ax.plot_surface(X, Y, p, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0, antialiased=False);
    
    return profile,


# How much we want to advance in time?
# at 30 fps if we want simulate 10 seconds => 30*10
t_seconds = 30 * 40;

# The time step size 
dt = 0.005;

partitions = 31;

data_x = np.linspace(0.0, 2.0, num=partitions, retstep=True);
x  = data_x[0]
dx = data_x[1];

data_y = np.linspace(0.0, 1.0, num=partitions, retstep=True);
y  = data_x[0]
dy = data_x[1];

X, Y = np.meshgrid(x, y);

#initial conditions
p = np.zeros((partitions, partitions));
"""
p = 0 at x = 0
p = y at x = 2
dp/dx = 0 at y = 0 and y = 1
"""
p[:,0] = 0;
p[:,-1] = y;
p[0,:] = p[1,:];
p[-1,:] = p[-2,:];

# Create a figure and a 3D Axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d');
ax.set_xlim(0,2)
ax.set_ylim(0,1)
ax.grid(False);
ax.view_init(azim=-150)
profile = ax.plot_surface(X, Y, p, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0, antialiased=False);

#Animate
anim = animation.FuncAnimation(fig, update, fargs=(profile,), frames=t_seconds, blit=False)
anim.save('2D_Laplace_Equation_00.mp4', fps=30, writer="ffmpeg", codec="libx264");
plt.close(fig);

In [5]:
# You must have configured ffmpeg in your machine to run the code below
from IPython.display import HTML
video = open("2D_Laplace_Equation_00.mp4", "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(video_tag)